# Probar Modelo

Pasos:

1. Colocar imagenes en la carpeta "02_probar_nuevas_fotos\fotos_prueba"

2. Ejecutar Notebook

definir librerias a instalar...

In [ ]:
# RECORTAR IMAGENES?
RECORTAR_IMAGENES = False

## Importar Librerías

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from PIL import Image
import cv2
import re
import face_recognition


## Importar objetos para predicción (modelo, pca, scaler)

In [3]:
import joblib
import pickle
scaler = joblib.load('scaler.pkl')
pca = joblib.load('pca.pkl')
label_binarizer = joblib.load('label_binarizer.pkl')
scaler_min_max = joblib.load('scaler_min_max.pkl')
cara_promedio_pca = joblib.load('cara_promedio_pca.pkl')
with open('weights_dict.pkl', 'rb') as archivo:
    weights_dict = pickle.load(archivo)


## Recortar imagenes

In [ ]:
data_dir = ("fotos_prueba")

## Definimos funcion para cortar caras de una imagen
def cortar_imagenes(input_dir, output_dir, dim=DIM):
    # Cargamos el detector de rostros de la libreria face_recognition
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    i = 0
    # Para cada archivo del directorio de imput comenzamos el loop
    for filename in os.listdir(input_dir):
        i += 1
        # Cargamos la imagen
        input_path = os.path.join(input_dir, filename)
        img = cv2.imread(input_path)

        # Detectamos el rostro en la imagen
        face_locations = face_recognition.face_locations(img)
        
        # Cortamos y cambiamos a escala de grises
        for (top, right, bottom, left) in face_locations:
            face = img[top:bottom, left:right]
            face = cv2.resize(face, (dim, dim))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            # Guardamos el proceso en la carpeta de salida
            output_path = os.path.join(output_dir, f"{filename}")
            cv2.imwrite(output_path, face)
            
if RECORTAR_IMAGENES:
    cortar_imagenes(input_dir="fotos_prueba", output_dir="fotos_prueba_recortadas")

## Importar imagenes recortadas

In [4]:
# Configuración inicial
data_dir = ("fotos_prueba_recortadas")  # Cambia esto a la ruta de tu directorio de imágenes
nombres = []
imagenes = []

# Cargar imágenes y etiquetas
for archivo in os.listdir(data_dir):
    if archivo.endswith('.jpeg') or archivo.endswith('.jpg'):
        nombre = archivo.split('_')[0].replace(".jpg","").replace(".jpeg","")
        nombre = re.sub(r'\d+', '', nombre)
        ruta_imagen = os.path.join(data_dir, archivo)
        imagen = Image.open(ruta_imagen)
        imagen = np.array(imagen.resize((30, 30)))  # Redimensionar imágenes para un tamaño uniforme
        if len(imagen.shape) == 3 and imagen.shape[2] == 3:
            imagen = cv2.cvtColor(imagen, cv2.COLOR_RGB2GRAY)  # Convertir a escala de grises si es necesario
        imagenes.append(imagen.flatten())
        nombres.append(nombre)

new_images_test = np.array(imagenes)
nombres_new_test = np.array(nombres)

# estand
new_images_test = new_images_test/255.0

In [6]:
greyscale_values_predict = pd.DataFrame(new_images_test)
people_names_predict = nombres_new_test.reshape(-1)

In [7]:
# definir X_new, nombres_y_new, y_new
X_new, nombres_y_new = greyscale_values_predict, people_names_predict
y_new = label_binarizer.transform(nombres_y_new)

In [8]:
print(X_new.shape)
nombres_y_new

(6, 900)


array(['lautaro', 'lautaro', 'lautaro', 'lautaro', 'paola', 'paola'],
      dtype='<U7')

In [11]:
# aplicar pca y escalado de datos

# obtener CPs
X_new_pca = pca.transform(X_new)

# estandarizar CPs
X_new_pca_scaled = scaler.transform(X_new_pca)

# escalamiento 0-1 CPs
X_new_pca_min_max = scaler_min_max.transform(X_new_pca)

# resto cara promedio
X_new_pca_menos_cara_promedio = X_new_pca - cara_promedio_pca

In [12]:
classes = label_binarizer.classes_

## Correr modelo seleccionado con las nuevas imagenes

In [13]:
from backpropagation_test import evaluate_predictions

In [14]:
# obtenemos predicciones
df_results_predict = evaluate_predictions(x_new = X_new_pca, y_labels=classes, y_value=np.array(nombres_y_new), weights_dict=weights_dict)

# mostrar bien clasificados
display(df_results_predict[df_results_predict["Nombre de la columna"]==df_results_predict["Nombre verdadero"]])

# mostrar mal clasificados
display(df_results_predict[df_results_predict["Nombre de la columna"]!=df_results_predict["Nombre verdadero"]])

# mostrar Error
(df_results_predict[df_results_predict["Nombre de la columna"]==df_results_predict["Nombre verdadero"]]).shape[0] / df_results_predict.shape[0] * 100

,Valor máximo,Nombre de la columna,Nombre verdadero
2,0.845194,lautaro,lautaro
4,0.997683,paola,paola
5,0.997663,paola,paola


,Valor máximo,Nombre de la columna,Nombre verdadero
0,0.013784,gerard,lautaro
1,0.077842,victorio,lautaro
3,0.944933,marco,lautaro


50.0